# Maps of the Iberian Peninsula

Olivier Defaux(1), Gerd Graßhoff(1,2), Mohammad Yeghaneh(2)

1: Max-Planck-Institute for the History of Science, Berlin; 
2: Humboldt University, Berlin

Date:

Ptolemy's catalogue of localities contains a list of places with their coordinates that enables to draw maps of the Iberian Peninsula. Ptolemy's text came down to us in two versions that are sometimes different regarding the spelling of the toponyms and the values of the coordinates. These versions are usually called "Omega recension" and "Xi recension". Based on the Greek text of Ptolemy's catalogue edited by A. Stückelberger and G. Graßhoff (2006), we will draw Ptolemy's map of the Iberian Peninsula and compare the two transmitted versions of his work.

In [1]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook, output_file, save
from bokeh.models import HoverTool,BoxZoomTool,ResetTool, WheelZoomTool
from bokeh.layouts import row, gridplot, layout

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)

In [4]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

In [5]:
from tools import Geography
from tools import reformatCoord,reformatIntFrac

Loading BokehJS ...

## Resources

We selected the part of Ptolemy's catalogue of localities related to the Iberian Peninsula (*Geography* 2.4–6) and we encoded the Greek text of the two recensions (Omega and Xi). The files can be loaded from the open-access plateform Zenodo.

### Import

In [6]:
# import .json file from DOI number

In [27]:
Omega = pd.read_json('../data/OmegaStructure2.json',encoding="utf8")
Xi = pd.read_json('../data/XiStructure2.json',encoding="utf8")

### Transform the .json files into dataframes

The .json files reproduce Ptolemy's text and structure. In order to work with the coordinates in an efficient way, we will transform the file format to produce a simple dataframe.

In [29]:
%run geographytools.py

In [30]:
df=Omega["chapters"][0]
om=json_normalize(df,"section")

In [31]:
def findTS(s):
    out=""
    for i,r in om.iterrows():
        sci=r["sec_ID"]
        if sci in s:
            out=r["type_sec"]
            
            exit
    return(out)
#findTS("2.04.01.01")

In [32]:
def Js2Geodf(df): 
# df=OmegaJ["chapters"][0]
    om=json_normalize(df,"section")
    om=om.dropna(subset=["sec_part"])
    l=[]
    for i,x in om.iterrows():
        k={"type_sec":x["type_sec"]}
        l.append([x["sec_part"]])
#
    listItems=list(flatten_list(l))
    dfout=pd.DataFrame(listItems)
    dfout["type_sec"]=dfout.apply(lambda x: findTS(x["ID"]),axis=1)
    return(dfout)

In [33]:
dfOmega=Js2Geodf(Omega["chapters"][0])
dfXi=Js2Geodf(Xi["chapters"][0])

In [34]:
# Sample of the dataframe for the Omega recension:
dfOmega[24:31]

,ID,category,coord,people,text,toponym,type,type_sec
24,2.04.05.13,city,"{'long': {'integer': 'ς', 'fraction': 'δ'}, 'lat': {'integer': 'λς', 'fraction': 'γο'}}",Turduli,NaN,Βαίλων πόλις,locality,coast section
25,2.04.06.01,NaN,NaN,NaN,Βαστουλῶν τῶν καλουμένων Ποινῶν,NaN,people,coast section
26,2.04.06.02,city,"{'long': {'integer': 'ς', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'L'}}",Bastuli,NaN,Μενραλία,locality,coast section
27,2.04.06.03,city,"{'long': {'integer': 'ς', 'fraction': 'γο'}, 'lat': {'integer': 'λς', 'fraction': 'γ'}}",Bastuli,NaN,Τρανσδούκτα,locality,coast section
28,2.04.06.04,city,"{'long': {'integer': 'ζ', 'fraction': 'δ'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Βαρβησόλα,locality,coast section
29,2.04.06.05,city,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Καρτμία,locality,coast section
30,2.04.06.06,mountain,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'δ'}}",Bastuli,NaN,Κάρπη ὅρος καὶ στήλη τῆς ἐντὸς θαλάσσης,locality,coast section


## Transform Greek numbers to decimal coordinates

### Greek numeral notation

Each latitude and each longitude is composed of two parts: an integer part (a whole number of degrees) and a decimal part (written as a fraction of degrees).
In order to plot the localities, we have to transpose the Greek numeral system used by Ptolemy into a modern format, using decimal. The symbols used by Ptolemy to express the coordinates are Greek letters and a special sign for the fraction 1/2.

In [35]:
gfrac={"":0,"ιβ":1/12,"ς":1/6,"δ":1/4,"γ":1/3,"γιβ":5/12,"L":1/2,"Lιβ":7/12,"γο":2/3,"Lδ":3/4,"Lγ":5/6,"Lγιβ":11/12,"η":1/8,"Lς":2/3,"ςL":2/3}
gint={"":0,"α":1,"β":2,"γ":3,"δ":4,"ε":5,"ς":6,"ζ":7,"η":8,"θ":9,"ι":10,"κ":20,"λ":30,"μ":40}

In [36]:
pd.DataFrame(list(gint.items()), columns=['Greek', 'Integers']).sort_values('Integers').set_index('Greek').T

Greek,,α,β,γ,δ,ε,ς,ζ,η,θ,ι,κ,λ,μ
Integers,0,1,2,3,4,5,6,7,8,9,10,20,30,40


In [37]:
pd.DataFrame(list(gfrac.items()), columns=['Greek', 'Decimal']).sort_values('Decimal').set_index('Greek').T

Greek,,ιβ,η,ς,δ,γ,γιβ,L,Lιβ,γο,Lς,ςL,Lδ,Lγ,Lγιβ
Decimal,0.0,0.083333,0.125,0.166667,0.25,0.333333,0.416667,0.5,0.583333,0.666667,0.666667,0.666667,0.75,0.833333,0.916667


### Translate into modern numeral notation

We create two columns for each dataframe: one for the longitude, one for the latitude.

In [38]:
# Omega
dfTemp = dfOmega.copy()
dfTemp['longitude'] = dfOmega.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(reformatIntFrac)
dfTemp['latitude'] = dfOmega.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(reformatIntFrac)

In [39]:
dfTemp[26:31]

,ID,category,coord,people,text,toponym,type,type_sec,longitude,latitude
26,2.04.06.02,city,"{'long': {'integer': 'ς', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'L'}}",Bastuli,NaN,Μενραλία,locality,coast section,6.500000,36.500000
27,2.04.06.03,city,"{'long': {'integer': 'ς', 'fraction': 'γο'}, 'lat': {'integer': 'λς', 'fraction': 'γ'}}",Bastuli,NaN,Τρανσδούκτα,locality,coast section,6.666667,36.333333
28,2.04.06.04,city,"{'long': {'integer': 'ζ', 'fraction': 'δ'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Βαρβησόλα,locality,coast section,7.250000,36.166667
29,2.04.06.05,city,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Καρτμία,locality,coast section,7.500000,36.166667
30,2.04.06.06,mountain,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'δ'}}",Bastuli,NaN,Κάρπη ὅρος καὶ στήλη τῆς ἐντὸς θαλάσσης,locality,coast section,7.500000,36.250000


In [67]:
# Xi
dfTempX = dfXi.copy()
dfTempX['longitude'] = dfXi.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(reformatIntFrac)
dfTempX['latitude'] = dfXi.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(reformatIntFrac)

## Data preparation for the map drawing

### Coasts and boundaries (Omega)

In [68]:
f = dfTemp[(dfTemp.type_sec == 'coast section') & (dfTemp.type == 'locality') & dfTemp.category.apply(lambda row: row not in ['boundary','river path','river source'])][['longitude','latitude']]

In [69]:
#fbae = coast of Baetica, omega
baria = dfTemp[dfTemp.toponym == 'ἐφʹ ἧς Βαρεία πόλις '][['longitude','latitude']]
fbae = f.iloc[0:28]
fbae = fbae.append(baria)
fbae = fbae.values

In [70]:
#flus = coast of Lusitania, omega
flus = f.iloc[28:42].append(f.iloc[0:1]).sort_index(ascending=True)
flus = flus.values

In [71]:
#ftar = coast of Tarraconensis, omega
ftar1 = f.iloc[42:70].append(f.iloc[41:42]).sort_index(ascending=True)
ftar1 = ftar1.values
ftar2 = f.iloc[70:].sort_index(ascending=False)
baebound = dfTemp[dfTemp.toponym == 'μετὰ τὸ πρὸς τῇ Βαιτικῇ πέρας'][['longitude','latitude']]
ftar2 = ftar2.append(baebound).append(baria)
ftar2 = ftar2.values

In [72]:
baebound

,longitude,latitude
257,12.0,37.25


In [73]:
### Coasts and boundaries (Xi)

In [74]:
g = dfTempX[(dfTempX.type_sec == 'coast section') & (dfTempX.type == 'locality') & dfTempX.category.apply(lambda row: row not in ['boundary','river path','river source'])][['longitude','latitude']]

In [75]:
#gbae = coast of Baetica, xi
bariaX = dfTempX[dfTempX.toponym == 'ἐφʹ ἧς Βαραλία πόλις'][['longitude','latitude']]
gbae = g.iloc[0:26]
gbae = gbae.append(bariaX)
gbae = gbae.values

In [76]:
#glus = coast of Lusitania, xi
glus = g.iloc[27:41].append(g.iloc[0:1]).sort_index(ascending=True)
glus = glus.values

In [77]:
#gtar = coast of Tarraconensis, xi
gtar1 = g.iloc[41:68].append(g.iloc[40:41]).sort_index(ascending=True)
gtar1 = gtar1.values
gtar2 = g.iloc[69:].sort_index(ascending=False)
baeboundX = dfTempX[dfTempX.toponym == 'μετὰ τὸ πρὸς τῇ Βαιτικῇ πέρας'][['longitude','latitude']]
gtar2 = gtar2.append(baeboundX).append(bariaX)
gtar2 = gtar2.values

In [78]:
# make a dictionary for all coasts's lines using Xi for later use in function plot_maps
gs_dict={'gbae' : gbae, 'glus' : glus, 'gtar1' : gtar1, 'gtar2' : gtar2}

In [79]:
# make a dictionary for all coasts's lines using Omega for later use in function plot_maps
fs_dict={'fbae' : fbae, 'flus' : flus, 'ftar1' : ftar1, 'ftar2' : ftar2}

In [80]:
# make a dictionary for all data
df_dict={"dfTemp" : dfTemp,  "dfTempX" : dfTempX}

We select th

## Map of the Iberian peninsula, Omega recension

In [81]:
fig=Geography(fs_dict,gs_dict,df_dict)

[map Omega]

In [82]:
fig.plot_recension(fs_dict,gs_dict,df_dict,select_recension='Omega')

## Map of the Iberian peninsula, Xi recension

[map Xi]

In [83]:
fig.plot_recension(fs_dict,gs_dict,df_dict,select_recension='Xi')

## Comparison between the two recensions

[map comparison]

In [84]:
fig.plot_recension_all(fs_dict,gs_dict,df_dict)